In [59]:
import pandas as pd
import numpy as np
import boto3
import logging
from botocore.exceptions import ClientError
import io
import os


In [64]:
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")


In [70]:
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region_name = 'eu-west-1'  # Replace with your AWS region
bucket_name = 'csv-files-parquet'
file_key = 'dummy_boat_data.csv'

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Read CSV file from S3 into a Pandas DataFrame
try:
    # Use 's3.get_object' to get the object and 'pd.read_csv' to read it into a DataFrame
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    df = pd.read_csv(obj['Body'])
    
    # Now 'df' contains your CSV data
    print(df.head())  # Display the first few rows of the DataFrame
except Exception as e:
    print(f"Error: {e}")

    Timestamp speed_over_ground    Longitude   Latitude engine_fuel_rate
0  1675119600              3.08  -119.605263  35.162237            14.92
1  1675119601              4.11  -118.857776  35.228734            21.02
2  1675119602              4.04  -118.110278  35.295227            20.53
3  1675119603               3.5  -117.362786  35.361725            17.21
4  1675119604             ERROR  -116.615295  35.428215            17.38
